In [1]:
import andes

In [2]:
!andes misc --version

Python  3.12.0
andes   1.9.3.post21.dev0+g37040896
numpy   2.2.5
kvxopt  1.3.2.1
sympy   1.14.0
scipy   1.13.1
pandas  2.2.3
numba   0.61.2


In [3]:
ss = andes.load("./Hawaii40_20231026.RAW",
                addfile="./Hawaii40_20231026.dyr",
                setup=False,
                no_output=True,)

Generating code for 1 models on 12 processes.


Error: <REPCA1> cannot retrieve <bus1> from <ACLine> using <line>:
  KeyError('Group <ACLine> does not contain device with idx=False')
Error: <REPCA1> cannot retrieve <bus2> from <ACLine> using <line>:
  KeyError('Group <ACLine> does not contain device with idx=False')
Error: <REPCA1> cannot retrieve <r> from <ACLine> using <line>:
  KeyError('Group <ACLine> does not contain device with idx=False')
Error: <REPCA1> cannot retrieve <x> from <ACLine> using <line>:
  KeyError('Group <ACLine> does not contain device with idx=False')


Manually set ``REPCA1.line``

In [4]:
idx = ss.REPCA1.idx.v
ree = ss.REPCA1.get(src='ree', attr='v', idx=idx)
reg = ss.REECA1.get(src='reg', attr='v', idx=ree)
bus = ss.REGCA1.get(src='bus', attr='v', idx=reg)
line1 = ss.Line.find_idx(keys='bus1', values=bus, allow_none=True, default=None)
line2 = ss.Line.find_idx(keys='bus2', values=bus, allow_none=True, default=None)
line = []
for l1, l2 in zip(line1, line2):
    if l1 is not None:
        line.append(l1)
    elif l2 is not None:
        line.append(l2)
    else:
        line.append(None)

ss.REPCA1.alter(src='line', attr='v', idx=idx, value=line)

In [5]:
ss.setup()

reg = ss.REECA1.get(src='reg', attr='v', idx=ss.REECA1.idx.v)
gen = ss.REGCA1.get(src='gen', attr='v', idx=reg)
vgen = ss.StaticGen.get(src='v0', attr='v', idx=gen)

ss.REECA1.alter(src='Vref1', attr='vin', idx=ss.REECA1.idx.v,
                value=vgen)

ss.REGCA1.alter(src='Iqrmax', attr='vin', idx=ss.REGCA1.idx.v,
                value=1)
ss.REGCA1.alter(src='Iqrmin', attr='vin', idx=ss.REGCA1.idx.v,
                value=-1)

ss.REECA1.alter(src='VMIN', attr='vin', idx=ss.REECA1.idx.v,
                value=-999)
ss.REECA1.alter(src='VMAX', attr='vin', idx=ss.REECA1.idx.v,
                value=999)
ss.REECA1.alter(src='QMin', attr='vin', idx=ss.REECA1.idx.v,
                value=-999)
ss.REECA1.alter(src='QMax', attr='vin', idx=ss.REECA1.idx.v,
                value=999)

ss.REECA1.alter(src='PMAX', attr='vin', idx=ss.REECA1.idx.v,
                value=999)
ss.REECA1.alter(src='Imax', attr='vin', idx=ss.REECA1.idx.v,
                value=999)

ss.REECA1.alter(src='Iql1', attr='vin', idx=ss.REECA1.idx.v,
                value=-999)
ss.REECA1.alter(src='Iqh1', attr='vin', idx=ss.REECA1.idx.v,
                value=999)

In [6]:
ss.PFlow.run()

  Multiple slack generators are defined/enabled for 1 island(s).


True

In [7]:
# ss.REGCA1.config.adjust_lower = 1
# ss.REECA1.config.adjust_lower = 1
# ss.REPCA1.config.adjust_lower = 1

_ = ss.TDS.init()

GENROU (vf range) out of typical upper limit.

   idx    | values | limit
----------+--------+------
 GENROU_1 | 21.304 | 5    
 GENROU_2 | 21.304 | 5    
 GENROU_3 | 21.304 | 5    
 GENROU_4 | 21.304 | 5    


/Users/jinningwang/.andes/pycode/REECA1.py:225: RuntimeWarning: invalid value encountered in divide
  return (-Iq1 + Iq2)/(-Vq1 + Vq2)
/Users/jinningwang/.andes/pycode/REECA1.py:229: RuntimeWarning: invalid value encountered in divide
  return (-Iq2 + Iq3)/(-Vq2 + Vq3)
/Users/jinningwang/.andes/pycode/REECA1.py:233: RuntimeWarning: invalid value encountered in divide
  return (-Iq3 + Iq4)/(-Vq3 + Vq4)
/Users/jinningwang/.andes/pycode/REECA1.py:241: RuntimeWarning: invalid value encountered in divide
  return (-Ip1 + Ip2)/(-Vp1 + Vp2)
/Users/jinningwang/.andes/pycode/REECA1.py:245: RuntimeWarning: invalid value encountered in divide
  return (-Ip2 + Ip3)/(-Vp2 + Vp3)
/Users/jinningwang/.andes/pycode/REECA1.py:249: RuntimeWarning: invalid value encountered in divide
  return (-Ip3

TDS initialization failed

In [8]:
andes.io.xlsx.write(ss, "./Hawaii40.xlsx", overwrite=True)

True